In this homework, we're going to combine data from various sources to process it in Pandas and generate additional fields.

If not stated otherwise, please use the [LINK][link] covered at the livestream to re-use the code snippets.

[link]: https://github.com/DataTalksClub/stock-markets-analytics-zoomcamp/blob/main/02-dataframe-analysis/%5B2025%5D_Module_02_Colab_Working_with_the_data.ipynb
---

# Question 1: [IPO] Withdrawn IPOs by Company Type

What is the total withdrawn IPO value (in $ millions) for the company class with the highest total withdrawal value?

From the withdrawn IPO list ([stockanalysis.com/ipos/withdrawn][wdipo]), collect and process the data to find out which company type saw the most withdrawn IPO value.
Steps:

1. Use `pandas.read_html()` with the URL above to load the IPO withdrawal table into a DataFrame.
   _It is a similar process to Code Snippet 1 discussed at the livestream._ You should get **99 entries**.
2. Create a new column called `Company Class`, categorizing company names based on patterns like:
    - "Acquisition Corp" or "Acquisition Corporation" → `Acq.Corp`
    - "Inc" or "Incorporated" → `Inc`
    - "Group" → `Group`
    - "Ltd" or "Limited" → `Limited`
    - "Holdings" → `Holdings`
    - Others → `Other`
   
   Order: Please follow the listed order of classes and assign the first matched value (e.g., for 'shenni holdings limited',
   you assign the `Limited` class).
   
    Hint: make your function more robust by converting names to lowercase and splitting into words before matching patterns.
4. Define a new field `Avg. price` by parsing the `Price Range` field (create a function and apply it to the `Price` Range column).
   Examples: 
   - '$8.00-$10.00' → `9.0`
   - '$5.00' → `5.0`
   - '-' → `None`
5. Convert Shares Offered to numeric, clean missing or invalid values.
6. Create a new column:
   `Withdrawn Value = Shares Offered * Avg Price` **(71 non-null values)**
7. Group by Company Class and calculate total withdrawn value.
8. **Answer**: Which class had the highest total value of withdrawals?

[wdipo]: https://stockanalysis.com/ipos/withdrawn/

In [1]:
from bs4 import BeautifulSoup
from polars import col as c
import polars as pl
import requests as r

def read_table(url: str) -> pl.DataFrame:
    resp = r.get(url)
    doc = BeautifulSoup(resp.content, 'html.parser')
    table = doc.find(id='main-table')
    keys = [e.text.strip() for e in table.find_all(name='th')]

    return pl.DataFrame([
        dict(zip(keys, [e.text for e in row.find_all(name='td')]))
        for row in table.find(name='tbody').find_all(name='tr')
    ])

In [2]:
df = read_table('https://stockanalysis.com/ipos/withdrawn')
df.shape

(100, 4)

In [3]:
df.head()

Symbol,Company Name,Price Range,Shares Offered
str,str,str,str
"""ODTX""","""Odyssey Therapeutics, Inc.""","""-""","""-"""
"""UNFL""","""Unifoil Holdings, Inc.""","""$3.00 - $4.00""","""2,000,000"""
"""AURN""","""Aurion Biotech, Inc.""","""-""","""-"""
"""ROTR""","""PHI Group, Inc.""","""-""","""-"""
"""ONE""","""One Power Company""","""-""","""-"""


In [4]:
df = (
    df.with_columns(
        c('Shares Offered').str.replace_all(',', '').str.to_integer(strict=False),
        (
            c('Price Range').str.replace_all(r'\$', '')
            .str.split(' - ')
            .list.eval(pl.element().cast(pl.Float32, strict=False))
            .list.mean()
        ).alias('Avg. price'),
        pl.when(c('Company Name').str.contains_any(['acquisition corp'], ascii_case_insensitive=True)).then(pl.lit('Acq.Corp'))
        .when(c('Company Name').str.contains_any(['inc'],                ascii_case_insensitive=True)).then(pl.lit('Inc'))
        .when(c('Company Name').str.contains_any(['group'],              ascii_case_insensitive=True)).then(pl.lit('Group'))
        .when(c('Company Name').str.contains_any(['ltd', 'limited'],     ascii_case_insensitive=True)).then(pl.lit('Limited'))
        .when(c('Company Name').str.contains_any(['holdings'],           ascii_case_insensitive=True)).then(pl.lit('Holdings'))
        .otherwise(pl.lit('Other'))
        .cast(pl.Categorical(ordering='lexical'))
        .alias('Company Class')
    )
    .with_columns(
        (c('Avg. price') * c('Shares Offered') / 1_000_000.0).alias('Withdrawn Value (million)')
    )
    .filter(c('Withdrawn Value (million)').is_not_null())
)
df

Symbol,Company Name,Price Range,Shares Offered,Avg. price,Company Class,Withdrawn Value (million)
str,str,str,i64,f32,cat,f64
"""UNFL""","""Unifoil Holdings, Inc.""","""$3.00 - $4.00""",2000000,3.5,"""Inc""",7.0
"""HPOT""","""The Great Restaurant Developme…","""$4.00 - $6.00""",1400000,5.0,"""Limited""",7.0
"""CABR""","""Caring Brands, Inc.""","""$4.00""",750000,4.0,"""Inc""",3.0
"""SQVI""","""Sequoia Vaccines, Inc.""","""$8.00 - $10.00""",2775000,9.0,"""Inc""",24.975
"""SNI""","""Shenni Holdings Limited""","""$4.00 - $6.00""",3000000,5.0,"""Limited""",15.0
…,…,…,…,…,…,…
"""DPAC""","""Deep Space Acquisition Corp. I""","""$10.00""",21000000,10.0,"""Acq.Corp""",210.0
"""GIF""","""GigCapital6, Inc.""","""$10.00""",20000000,10.0,"""Inc""",200.0
"""HYIV""","""Haymaker Acquisition Corp. IV""","""$10.00""",26100000,10.0,"""Acq.Corp""",261.0


In [5]:
pl.Config.set_float_precision(2)

(
    df.group_by('Company Class')
    .agg(c('Withdrawn Value (million)').sum())
).sort('Withdrawn Value (million)', descending=True)

Company Class,Withdrawn Value (million)
cat,f64
"""Acq.Corp""",4021.00
"""Inc""",2257.16
"""Other""",767.92
"""Limited""",549.73
"""Holdings""",75.00
"""Group""",33.79


# Question 2: [IPO] Median Sharpe Ratio for 2024 IPOs (First 5 Months)

What is the median Sharpe ratio (as of 6 June 2025) for companies that went public in the first 5 months of 2024?

The goal is to replicate the large-scale yfinance OHLCV data download and perform basic financial calculations on IPO stocks.

Steps:

1. Using the same approach as in Question 1, download the IPOs in 2024 from: https://stockanalysis.com/ipos/2024/
   Filter to keep only those IPOs before 1 June 2024 (first 5 months of 2024). ➤ You should have 75 tickers.
2. Use Code Snippet 7 to download daily stock data for those tickers (via yfinance).
   Make sure you understand how `growth_1d` ... `growth_365d`, and volatility columns are defined.
   Define a new column `growth_252d` representing growth after 252 trading days (~1 year),
   in addition to any other growth periods you already track.
3. Calculate the Sharpe ratio assuming a risk-free rate of 4.5%:

    stocks_df['Sharpe'] = (stocks_df['growth_252d'] - 0.045) / stocks_df['volatility']

4. Filter the DataFrame to keep data only for the trading day: '2025-06-06'.
   Compute descriptive statistics (e.g., .describe()) for these columns:
   - `growth_252`
   - Sharpe
   You should observe:
   - `growth_252d` is defined for 71 out of 75 stocks (some IPOs are too recent or data starts later).
   - Median `growth_252d` is approximately 0.75 (indicating a 25% decline), while mean is about 1.15,
     showing a bias towards high-growth companies pushing the average up.

5. Answer:
   - What is the median Sharpe ratio for these 71 stocks?
   - Note: Positive Sharpe means growth exceeding the risk-free rate of 4.5%.
   - [Additional] Do you observe the same top 10 companies when sorting by `growth_252d` versus sorting by Sharpe?

In [6]:
df = read_table('https://stockanalysis.com/ipos/2024/')
df

IPO Date,Symbol,Company Name,IPO Price,Current,Return
str,str,str,str,str,str
"""Dec 31, 2024""","""ONEG""","""OneConstruction Group Limited""","""$4.00""","""$3.73""","""-6.64%"""
"""Dec 27, 2024""","""PHH""","""Park Ha Biological Technology …","""$4.00""","""$19.80""","""395.00%"""
"""Dec 23, 2024""","""HIT""","""Health In Tech, Inc.""","""$4.00""","""$0.60""","""-85.08%"""
"""Dec 23, 2024""","""TDAC""","""Translational Development Acqu…","""$10.00""","""$10.28""","""2.80%"""
"""Dec 20, 2024""","""RANG""","""Range Capital Acquisition Corp…","""$10.00""","""$10.31""","""3.10%"""
…,…,…,…,…,…
"""Jan 18, 2024""","""CCTG""","""CCSC Technology International …","""$6.00""","""$1.08""","""-82.00%"""
"""Jan 18, 2024""","""PSBD""","""Palmer Square Capital BDC Inc.""","""$16.45""","""$14.20""","""-13.68%"""
"""Jan 12, 2024""","""SYNX""","""Silynxcom Ltd.""","""$4.00""","""$1.89""","""-52.75%"""


In [7]:
from datetime import date

symbols = (
    df.with_columns(
        c('IPO Date').str.strptime(pl.Date, '%B %d, %Y'),
        c('IPO Price').str.replace(r'\$', '').cast(pl.Float32, strict=False),
        c('Current').str.replace(r'\$', '').cast(pl.Float32, strict=False),
        c('Return').str.replace('%', '').cast(pl.Float32, strict=False),
    )
    .filter((c('IPO Date') < date(2024, 6, 1)) & c('Return').is_not_null())
)['Symbol'].to_list()

', '.join(symbols)

'BOW, HDL, RFAI, JDZG, RAY, BTOC, ZK, GPAT, PAL, SVCO, NNE, CCIX, VIK, ZONE, LOAR, MRX, RBRK, NCI, MFI, YYGH, TRSG, CDTG, CTRI, IBTA, MTEN, TWG, ULS, PACS, MNDR, CTNM, MAMO, ZBAO, BOLD, MMA, UBXG, IBAC, AUNA, BKHA, LOBO, RDDT, ALAB, INTJ, RYDE, LGCL, SMXT, VHAI, DYCQ, CHRO, UMAC, HLXB, MGX, TBBB, TELO, KYTX, PMNT, AHR, LEGT, ANRO, GUTS, AS, FBLG, AVBP, BTSG, HAO, CGON, YIBO, JL, SUGP, JVSA, KSPI, CCTG, PSBD, SYNX, SDHC, ROMA'

In [8]:
import yfinance as yf
from pathlib import Path

def load_ohlc_history(symbol: str) -> pl.DataFrame:
    fname = Path.cwd() / f'cache/{symbol.lower()}.csv'
    
    if fname.exists():
        df = (
            pl.read_csv(fname)
            .with_columns(
                Date=c('Date').str.strptime(pl.Datetime(time_unit='us'), '%Y-%m-%dT%H:%M:%S%.9f%z', strict=False),
            )
        )
    else:
        df = pl.DataFrame(yf.Ticker(symbol).history(period='max', interval='1d').reset_index())
        df.write_csv(fname)   # cache the data to prevent API throttling problems during experimentations
    
    return df.with_columns(
        Date=c('Date').dt.convert_time_zone('Etc/UTC').dt.date(),
        Symbol=pl.lit(symbol),
    ).sort('Date', descending=False)

In [9]:
from tqdm.auto import tqdm
import numpy as np

df = (
    pl.concat([load_ohlc_history(symbol) for symbol in tqdm(symbols)])
    .lazy()
    .select('Symbol', 'Date', 'Close')
    .sort('Symbol', 'Date')
    .collect()
).lazy()
df.collect()

  0%|          | 0/75 [00:00<?, ?it/s]

Symbol,Date,Close
str,date,f64
"""AHR""",2024-02-07,12.43
"""AHR""",2024-02-08,12.28
"""AHR""",2024-02-09,12.34
"""AHR""",2024-02-12,12.37
"""AHR""",2024-02-13,12.56
…,…,…
"""ZONE""",2025-06-09,3.89
"""ZONE""",2025-06-10,3.79
"""ZONE""",2025-06-11,3.72


In [10]:
(
    df.with_columns(
        growth_252d=c('Close').over('Symbol') / c('Close').shift(252).over('Symbol'),
        volatility=c('Close').rolling_std(30) * np.sqrt(252),
    )
    .with_columns(
        Sharpe=c('growth_252d') / c('volatility')
    )
    .filter(c('Date') == date(2025, 6, 6))
    .select(
        growth_252d_median=c('growth_252d').filter(c('growth_252d').is_not_nan() & c('growth_252d').is_not_null()).median(),
        non_null_growth_252d=c('growth_252d').is_not_null().count(),
        non_nan_growth_252d=c('growth_252d').is_not_nan().count(),
        non_null_sharpe=c('Sharpe').is_not_null().count(),
        non_nan_sharpe=c('Sharpe').is_not_nan().count(),
        sharpe_median=c('Sharpe').filter(c('Sharpe').is_not_nan() & c('Sharpe').is_not_null()).median(),
    )
    .collect()
).transpose(include_header=True, header_name='Measure', column_names=['Value'])

Measure,Value
str,f64
"""growth_252d_median""",0.76
"""non_null_growth_252d""",75.00
"""non_nan_growth_252d""",71.00
"""non_null_sharpe""",75.00
"""non_nan_sharpe""",71.00
"""sharpe_median""",0.10


# Question 3: [IPO] ‘Fixed Months Holding Strategy’

What is the optimal number of months (1 to 12) to hold a newly IPO'd stock in order to maximize average growth?
_(Assume you buy at the close of the first trading day and sell after a fixed number of trading days.)_

---

Goal:

Investigate whether holding an IPO stock for a fixed number of months after its 
first trading day produces better returns, using future growth columns.

---

Steps:

1. Start from the existing DataFrame from Question 2 (75 tickers from IPOs in the first 5 months of 2024).

   Add 12 new columns:

   `future_growth_1m`, `future_growth_2m`, ..., `future_growth_12m`

    _(Assume 1 month = 21 trading days, so growth is calculated over 21, 42, ..., 252 trading days)_

    This logic is similar to `historyPrices['growth_future_30d']` from Code Snippet 7, but extended to longer timeframes.

2. Determine the first trading day (min_date) for each ticker.

   This is the earliest date in the data for each stock.

3. Join the data:

   Perform an inner join between the `min_date` DataFrame and the future growth data on both `ticker` and `date`.

   ➤ You should end up with 75 records (one per IPO) with all 12 `future_growth_...` fields populated.

4. Compute descriptive statistics for the resulting DataFrame:

   Use `.describe()` or similar to analyze each of the 12 columns:

   - `future_growth_1m`
   - `future_growth_2m`
   - ...
   - `future_growth_12m`

5. Determine the best holding period:

   - Find the number of months (1 to 12) where the average (mean) future growth is maximal.
   - This optimal month shows an uplift of >1% compared to all others.
   - Still, the average return remains less than 1 (i.e., expected return is less than doubling your investment).

In [11]:
pl.Config.set_tbl_rows(12)

(
  df.with_columns(
      *[(c('Close').shift(-21 * (i + 1)).over('Symbol') / c('Close').over('Symbol')).alias(f'future_growth_{i + 1}m') for i in range(12)]
  )
  .group_by('Symbol')
  .agg(pl.all().head(1))
  .explode(pl.exclude('Symbol'))
  .select(
      *[c(f'future_growth_{i + 1}m').mean().alias(f'{i + 1}') for i in range(12)]
  )
  .collect()
  .transpose(include_header=True, header_name='num_of_months', column_names=['average_future_growth'])
  .with_columns(num_of_months=c('num_of_months').str.to_integer())
).sort('average_future_growth', descending=True)

num_of_months,average_future_growth
i64,f64
2,0.94
1,0.93
10,0.92
12,0.90
11,0.88
9,0.88
6,0.86
7,0.85
3,0.83


# Question 4: [Strategy] Simple RSI-Based Trading Strategy

What is the total profit (in $thousands) you would have earned by investing $1000 every time a stock was oversold (RSI < 25)?
--- 
Goal:

Apply a simple rule-based trading strategy using the Relative Strength Index (RSI) technical indicator to identify oversold
signals and calculate profits.
---

Steps:

1. Run the full notebook from Lecture 2 (33 stocks)
   - Ensure you can generate the merged DataFrame containing:
     - OHLCV data
     - Technical indicators
    - Macro indicators
   - Focus on getting RSI computed using Code Snippets 8 and 9.
   - This process is essential and will help during the capstone project.
2. ⚠️ IMPORTANT Please use this file to solve the Home Assignment (all next steps)

   Download precomputed data using this snippet:

```python
import gdown
import pandas as pd

file_id = "1grCTCzMZKY5sJRtdbLVCXg8JXA8VPyg-"
gdown.download(f"https://drive.google.com/uc?id={file_id}", "data.parquet", quiet=False)
df = pd.read_parquet("data.parquet", engine="pyarrow")
```

3. RSI Strategy Setup:
   - RSI is already available in the dataset as a field.
   - The threshold for oversold is defined as RSI < 25.
4. Filter the dataset by RSI and date:

```python
rsi_threshold = 25
selected_df = df[
    (df['rsi'] < rsi_threshold) &
    (df['Date'] >= '2000-01-01') &
    (df['Date'] <= '2025-06-01')
```

5. Calculate Net Profit Over 25 Years:
   - Total number of trades: 1568
   - For each trade, you invest $1000
   - Use the 30-day forward return (growth_future_30d) to compute net earnings:

```python
net_income = 1000 * (selected_df['growth_future_30d'] - 1).sum()
```
   - Final Answer:

     What is the net income in $K (i.e., in thousands of dollars) that could be earned using this
     RSI-based oversold strategy from 2000–2025?

In [12]:
def load_homework_data() -> pl.DataFrame:
    fname = Path.cwd() / 'cache/data.parquet'

    if not fname.exists():
        file_id = '1grCTCzMZKY5sJRtdbLVCXg8JXA8VPyg-'
        url = f'https://drive.usercontent.google.com/download?id={file_id}&export=download&confirm=yes'
        content = r.get(url).content
        with open(fname, 'wb') as fout:
            fout.write(content)

    return pl.read_parquet(fname)

df = load_homework_data()
df

Open,High,Low,Close_x,Volume,Dividends,Stock Splits,Ticker,Year,Month,Weekday,Date,growth_1d,growth_3d,growth_7d,growth_30d,growth_90d,growth_365d,growth_future_30d,SMA10,SMA20,growing_moving_average,high_minus_low_relative,volatility,is_positive_growth_30d_future,ticker_type,index_x,adx,adxr,apo,aroon_1,aroon_2,aroonosc,bop,cci,cmo,dx,…,growth_epi_90d,growth_epi_365d,Quarter,gdppot_us_yoy,gdppot_us_qoq,cpi_core_yoy,cpi_core_mom,FEDFUNDS,DGS1,DGS5,DGS10,Close_y,growth_gold_1d,growth_gold_3d,growth_gold_7d,growth_gold_30d,growth_gold_90d,growth_gold_365d,growth_wti_oil_1d,growth_wti_oil_3d,growth_wti_oil_7d,growth_wti_oil_30d,growth_wti_oil_90d,growth_wti_oil_365d,growth_brent_oil_1d,growth_brent_oil_3d,growth_brent_oil_7d,growth_brent_oil_30d,growth_brent_oil_90d,growth_brent_oil_365d,growth_btc_usd_1d,growth_btc_usd_3d,growth_btc_usd_7d,growth_btc_usd_30d,growth_btc_usd_90d,growth_btc_usd_365d,__index_level_0__
f64,f64,f64,f64,f64,f64,f64,str,i32,datetime[ns],i32,datetime[ns],f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,i64,str,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,datetime[ns],f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64
0.05,0.06,0.05,0.06,1031788800.00,0.00,0.00,"""MSFT""",1986,1986-03-01 00:00:00,3,1986-03-13 00:00:00,null,null,null,null,null,null,0.98,null,null,0,0.13,null,0,"""US""",0,null,null,null,null,null,null,0.67,null,null,null,…,null,null,1986-01-01 00:00:00,0.04,0.01,0.04,0.00,7.48,7.07,7.46,7.75,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,0
0.06,0.06,0.06,0.06,308160000.00,0.00,0.00,"""MSFT""",1986,1986-03-01 00:00:00,4,1986-03-14 00:00:00,1.04,null,null,null,null,null,0.92,null,null,0,0.05,null,0,"""US""",1,null,null,null,null,null,null,0.67,null,null,null,…,null,null,1986-01-01 00:00:00,0.04,0.01,0.04,0.00,7.48,7.00,7.40,7.72,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1
0.06,0.06,0.06,0.06,133171200.00,0.00,0.00,"""MSFT""",1986,1986-03-01 00:00:00,0,1986-03-17 00:00:00,1.02,null,null,null,null,null,0.88,null,null,0,0.03,null,0,"""US""",2,null,null,null,null,null,null,0.67,null,null,null,…,null,null,1986-01-01 00:00:00,0.04,0.01,0.04,0.00,7.48,7.03,7.42,7.77,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,2
0.06,0.06,0.06,0.06,67766400.00,0.00,0.00,"""MSFT""",1986,1986-03-01 00:00:00,1,1986-03-18 00:00:00,0.97,1.03,null,null,null,null,0.92,null,null,0,0.04,null,0,"""US""",3,null,null,null,null,null,null,-0.60,null,null,null,…,null,null,1986-01-01 00:00:00,0.04,0.01,0.04,0.00,7.48,7.03,7.48,7.83,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,3
0.06,0.06,0.06,0.06,47894400.00,0.00,0.00,"""MSFT""",1986,1986-03-01 00:00:00,2,1986-03-19 00:00:00,0.98,0.97,null,null,null,null,0.96,null,null,0,0.04,null,0,"""US""",4,null,null,null,null,null,null,-0.50,null,null,null,…,null,null,1986-01-01 00:00:00,0.04,0.01,0.04,0.00,7.48,7.02,7.46,7.82,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,4
0.06,0.06,0.06,0.06,58435200.00,0.00,0.00,"""MSFT""",1986,1986-03-01 00:00:00,3,1986-03-20 00:00:00,0.97,0.93,null,null,null,null,1.01,null,null,0,0.04,null,1,"""US""",5,null,null,null,null,null,null,-0.75,null,null,null,…,null,null,1986-01-01 00:00:00,0.04,0.01,0.04,0.00,7.48,7.00,7.45,7.78,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,5
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
3570.00,3631.60,3551.20,3599.90,1119217.00,0.00,0.00,"""LT.NS""",2025,2025-05-

In [13]:
(
    df.lazy()
    .with_columns(
        Date=c('Date').dt.date(),
        income=(c('growth_future_30d') - 1) * 1000
    )
    .filter((c('rsi') < 25) & (c('Date') >= date(2000, 1, 1)) & (c('Date') <= date(2025, 6, 1)))
    .select(
        net_income_thousands=c('income').sum() / 1000.0,
        num_trades=pl.first().count(),
    )
).collect()

net_income_thousands,num_trades
f64,u32
24.30,1568
